# Rafal Load stress testing

## Connect to Rafal

In [ ]:
# load connection script
%run ../../etc/rafal_load.py

uiConnect

## Parse log file and look for rafal requests

In [ ]:
log_file= 'log.txt'

In [ ]:
def tokenize(line):
    return line.strip().split(' ')

In [ ]:
verbose= True

In [ ]:
read_request= False
requestsArgs= []

with open(log_file, 'r') as fin:
    for i, line in enumerate(fin.readlines()):
        if read_request and (line[:3] == '202'):
            # end of endPoint request detected
            requestsArgs.append({'httpMethod': httpMethod,
                                 'endPoint':endPoint,
                                 'body': ''.join(body),
                                })
            if verbose:
                print(i, httpMethod, endPoint, len(body))
            read_request = False
            
        if read_request:
            # read body line
            body.append(line)
            
        if (line[:3] == '202') and ("DEBUG - TRAFFIC." in line):
            # new endPoint request detected
            read_request= True
            cmd= tokenize(line)
            httpMethod= cmd[6]
            endPoint= cmd[7]
            if len(cmd) > 8 and cmd[8] == '{':
                body= [cmd[8]]
            else:
                body= []

## replay rafal request

In [ ]:
import json

In [ ]:
for i, requestArgs in enumerate(requestsArgs):
    if requestArgs['endPoint'] == '/ping':
        print('/ping');continue
    print('\n', i, requestArgs['httpMethod'], requestArgs['endPoint'])
    requestArgs['resp']= session.request(endPoint= requestArgs['endPoint'], 
                                         http= requestArgs['httpMethod'], 
                                         output= 'json',
                                         jsonText= json.loads(requestArgs['body']) if requestArgs['body'] else None)

In [ ]:
for requestArgs in requestsArgs:
    if 'resp' in requestArgs:
        print(requestArgs['httpMethod'], requestArgs['endPoint'], len(requestArgs['resp']) if requestArgs['resp'] else 0)